## Twitter Sentimental Analysis 

We would be analaysing how 1,000 random Nigerians feel about the commercial banks and digital banks they use.

### Workflow 

* Create Twitter Developer account 
* Scrape the data from Twitter 
* Clean tweets 
* Analyze the Data 
* Data Visualizaton 
* Hypothesis 


In [7]:
! pip install tweepy

  Using cached https://files.pythonhosted.org/packages/67/c3/6bed87f3b1e5ed2f34bd58bf7978e308c86e255193916be76e5a5ce5dfca/tweepy-3.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a3/12/b92740d845ab62ea4edf04d2f4164d82532b5a0b03836d4d4e71c6f3d379/requests_oauthlib-1.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/57/ce2e7a8fa7c0afb54a0581b14a65b56e62b5759dbc98e80627142b8a3704/oauthlib-3.1.0-py2.py3-none-any.whl


In [13]:
!pip install textblob

In [133]:
!pip install WordCloud

In [2]:
#import dependencies 
import tweepy
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from textblob import TextBlob 
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
plt.style.use("fivethirtyeight")

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
wordLemm = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to C:\Users\TOYIN
[nltk_data]     OLAPE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\TOYIN
[nltk_data]     OLAPE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#Access Twitter API  

key  =  "MfWW96NNbgC2BP9gYvW0xNf3z"
secret = "et5tSv0hq1xQbcZwgqgA7tQSK2xnvvPcFoU9ofihqf98cvyy7u"

access_token = "366459074-DYjx7MdizB10YVQuxIVwBRniKvcASx6EKMZogHn6"
access_token_secret = "j50GFMWpvpvGInQudjRMm6mAnM6Unm7gBjlWpov99Tq01"


In [5]:
#Authenticate Twitter dev. access token
auth = tweepy.OAuthHandler(key, secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

## Scope of Project
General Sentimental Analysis of Commercial Banks and Digital Banks in Nigeria. An indepth analysis on the 5 biggest Commercial Banks in Nigeria. 


## KUDA BANK 

In [6]:
#Parse twitter for Kuda bank tweets 

kuda_list = []

for tweet in tweepy.Cursor(api.search, q='kudabank-filter:retweets', language = "en", tweet_mode='extended',geocode ='10,7,1000mi').items(1500):
    kuda_list.append(tweet.full_text)


df_kuda = pd.DataFrame(kuda_list, columns=['Tweets'])
df_kuda.drop_duplicates(inplace = True)

In [7]:
df_kuda.shape

(1412, 1)

In [8]:
df_kuda.head()

,Tweets
0,@OwoborodeBiodun @GbengaGOLD @Babsogundeyi @Ni...
1,@Alexcassy @FS_AYODELE @GbengaGOLD @Babsogunde...
2,@FS_AYODELE @OwoborodeBiodun @GbengaGOLD @Babs...
3,@kudabank If you live in owerri like this twét
4,"@kudabank pls help with my account password , ..."


We can see that we have so much useless data to remove it 

we need to clean the data

for cleaning we are going to use `re` (regex) 

process : 
    1. Lower Casing: Each text is converted to lowercase.
    2. Replacing URLs: Links starting with "http" or "https" or "www" are replaced by "URL".
    3. Replacing Emojis: Replace emojis by using a pre-defined dictionary containing emojis along with their meaning. (eg: ":)" to "EMOJIsmile")
    4. Replacing Usernames: Replace @Usernames with word "USER". (eg: "@Kaggle" to "USER")
    5. Removing Non-Alphabets: [^a-zA-z]
    6. Removing Consecutive letters: 3 or more consecutive letters are replaced by 2 letters. (eg: "Heyyyy" to "Heyy")
    7. Removing Short Words: Words with length less than 2 are removed.
    8. Removing Stopwords: Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. (eg: "the", "he", "have")
    9. Lemmatizing: Lemmatization is the process of converting a word to its base form. (e.g: “Great” to “Good”)

In [10]:
EMOJIS = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}
URLPATTERN        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
USERPATTERN       = '@[^\s]+'
SEQPATTERN   = r"(.)\1\1+"
SEQREPLACE = r"\1\1"

In [11]:
corpus_kuda = []
for texts in df_kuda["Tweets"]:
    
    text = texts.lower()
    ### Replacing URL
    text = re.sub(URLPATTERN,' URL',text)
    ### Replacing EMOJI
    for emoji in EMOJIS.keys():
        text = text.replace(emoji, "EMOJI" + EMOJIS[emoji])  
    ### Replacing USER pattern
    text = re.sub(USERPATTERN,' URL',text)
    ### Removing non-alphabets
    text = re.sub('[^a-zA-z]'," ",text)
    ### Removing consecutive letters
    text = re.sub(SEQPATTERN,SEQREPLACE,text)
    text = text.split()
    text = [wordLemm.lemmatize(word) for word in text if not word in stopwords.words('english') and len(word) > 1]
    text = ' '.join(text)
    corpus_kuda.append(text)

corpus_kuda[1]


'URL URL URL URL URL URL URL URL mi ni kuda account abi wa fun mi ni owo ti fi si kuda account'

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

CV = CountVectorizer(max_features=3000,ngram_range=(1,2))
df_kuda = CV.fit_transform(corpus_kuda).toarray()

df_kuda.shape

(1412, 3000)

## Building Functions

In [47]:
#Since i am going to parse for several banks. I will go aheadd and change the above code to a function
def parse_tweet(keyword):
  listname = []

  for tweet in tweepy.Cursor(api.search, q= keyword + '-filter:retweets', language = "en",tweet_mode='extended',geocode ='10,7,1000mi' ).items(1500):
    listname.append(tweet.full_text)
  
  return listname
     

In [108]:
#Similarly, i will make the cleaning function in order to cleaning the other data sets 
import time
def clean_tweets(listname):
    
    t = time.time()
    corpus = []
    for texts in listname:
        ## lower casing
        text = texts.lower()
        ### Replacing URL
        text = re.sub(URLPATTERN,' URL',text)
        ### Replacing EMOJI
        for emoji in EMOJIS.keys():
            text = text.replace(emoji, "EMOJI" + EMOJIS[emoji])  
        ### Replacing USER pattern
        text = re.sub(USERPATTERN,' URL',text)
        ### Removing non-alphabets
        text = re.sub('[^a-zA-z]'," ",text)
        ### Removing consecutive letters
        text = re.sub(SEQPATTERN,SEQREPLACE,text)
        text = text.split()
        text = [wordLemm.lemmatize(word) for word in text if not word in stopwords.words('english') and len(word) > 1]
        text = ' '.join(text)
        corpus.append(text)
    return corpus
    print(f'Time Taken: {round(time.time()-t)} seconds')

In [ ]:
def bag_of_words(corpus):

    CV = CountVectorizer(max_features=3000,ngram_range=(1,2))
    df_1 = CV.fit_transform(corpus).toarray()
    return df_1

## GTB Bank 

In [112]:
gtb_list = parse_tweet("gtb")

df_gtb = pd.DataFrame(gtb_list1, columns=['Tweets'])
df_gtb.drop_duplicates(inplace = True)
df_gtb.head()

,Tweets
0,"If Beyonce was born in Nigeria in 1981, there'..."
1,@kayzywizzzy 0210132521 gtb
2,Most capitalized companies on NSE as at 19th M...
3,@cuppymusic @BetKingNG In case the money show....
4,@crayonthis Dm not open\n\nGtb\n0414086570\n🙌😍...


In [113]:
df_gtb.shape

(1500, 1)

In [110]:
corpus_gtb = clean_tweets('df_gtb["Tweets"]')
corpus_gtb[300]

IndexError: list index out of range

## First Bank 

In [51]:
fb_list = parse_tweet("firstbank")

df_fb = pd.DataFrame(fb_list, columns=['Tweets'])
df_fb.drop_duplicates(inplace = True)
df_fb.head()

,Tweets
0,@cuppymusic @BetKingNG Aunty cuppy no fear we ...
1,@Mbahdeyforyou I for send the remaining #100 t...
2,@Abbeytips101 I don drop Aza 3102072331 firstbank
3,"You see how cute the iPhone 12 is, one person ..."
4,Try out the new FirstMobile App from FirstBank...


In [ ]:
corpus_fb = clean_tweets('df_fb["Tweets"]')
corpus_fb[300]

## Zenith Bank

In [52]:
zb_list = parse_tweet("zenithbank")

df_zb = pd.DataFrame(zb_list, columns=['Tweets'])
df_zb.drop_duplicates(inplace = True)
df_zb.head()

,Tweets
0,Top 10 Banks on the NSE by Market Cap as at 19...
1,@JohnsonDicksonO @ZenithDirect_ Kindly be info...
2,@zenithbank91 Kindly be informed that we prese...
3,@_AsiwajuLerry Omo e belike u go send me money...
4,@Duki11223879 @cenbank Please remember that Ze...


In [ ]:
corpus_zb = clean_tweets('df_zb["Tweets"]')
corpus_zb[300]

## United Bank of Africa

In [114]:
uba_list1 = parse_tweet("UBA")

df_uba2 = pd.DataFrame(uba_list1, columns=['Tweets'])
df_uba2.drop_duplicates(inplace = True)
df_uba2.head()


,Tweets
0,@Toshibasexy @fkeyamo I have not received any ...
1,@Irunnia_ Boss me dy always dy active\n🙏🏾💯\n21...
2,@blavk_pharouqq Kace bara'uba kawai
3,"@Irunnia_ Bless me sir this night ,2163017201 ..."
4,@Irunnia_ 2145013046\nUBA


In [ ]:
corpus_uba = clean_tweets('df_uba2["Tweets"]')
corpus_uba[300]

## Access Bank

In [56]:
ab_list = parse_tweet("AccessBank")

df_ab = pd.DataFrame(ab_list, columns=['Tweets'])
df_ab.drop_duplicates(inplace = True)
df_ab.head()

,Tweets
0,@accessbank_help kindly refer to my DMs that I...
1,"@Jaru0 Hello Jaru0, we have replied your Dm. K..."
2,"@pajoil25 Hi pajoil25, please check your DM. \..."
3,@accessbank_help please check your DM thanks
4,"@EmekaBennard Hello @EmekaBennard, thanks for ..."


In [ ]:
corpus_ab = clean_tweets('df_ab["Tweets"]')
corpus_ab[300]

## Assumptions & Challenges 

The data collected for first bank would also be quite messy, because the word "first" can be found with other banks and therefore it means there would be  lot of noise in the data set collected for First bank. 

Collecting the data for Kuda Bank at this time is heavily influenece by the fact that they just recieved a Series A funding of $25 million, Majority of the tweets containing this keyword would be congratolatory and therefore Positive 

Nigeria is currently the poverty capital of the world and as such most of the tweets are of people posting their account number in order to recieve funds from kind strangers.


In [ ]:
#clean the text
def cleantxt(text):
  text = re.sub(r'@')

## Further Work 

Traingulate the tweets according to there locations to better understand the areass with negative tweets in order to improve bank services and improve the overall positive sentiment. 

Filter out give away tweets to reduce the noise 